In [ ]:
import os
import glob
import numpy as np
import xarray as xr

os.chdir('../')
project_dir = os.getcwd()

from resow import *
Config = ReSOW_Config()

In [ ]:
data_directory = os.path.join(Config.PATHS['data partition'], Config.PATHS['tile'])

if os.path.isdir(data_directory):
    printProgress(f'data directory is: "{data_directory}"')
else:
    printError(f'check for valid data directory in "PATHS": {data_directory}')
    
netCDF_filepaths = glob.glob(pathname=data_directory+'/*.nc',
                             recursive=False)

printProgress(f'found {len(netCDF_filepaths)} netCDF files')

In [ ]:
for index, netCDF_file in enumerate(netCDF_filepaths):    
    printProgress(f'processing {netCDF_file}')

    data_xr = xr.open_dataset(netCDF_file)
    x_attrs = data_xr.coords['x'].attrs
    y_attrs = data_xr.coords['y'].attrs
    
    for key in data_xr.keys():    
        if 'B' in key:
            printProgress(f'{key} {np.min(data_xr[key]):.2f}' +
                          f' {np.mean(data_xr[key]):.2f}' +
                          f' {np.max(data_xr[key]):.2f}' +
                          f' {np.count_nonzero(data_xr[key]<0)}')
            new_da = xr.where(data_xr[key] < 0, -999, data_xr[key],
                             keep_attrs=True)
            new_da.attrs = data_xr[key].attrs
            data_xr[key] = new_da
    
    data_xr.coords['x'].attrs = x_attrs
    data_xr.coords['y'].attrs = y_attrs
    
    output_filepath = netCDF_file.replace('.nc', '_pp.nc')
    printProgress(f'writing: {output_filepath}')
    data_xr.to_netcdf(output_filepath, mode="w", format="NETCDF4")
    printProgress('')
    
printProgress('done')

In [ ]:
data_xr['B11'].plot()

In [ ]:
data_xr.keys()